In [22]:
import os
import xarray as xr
import pandas as pd
import numpy as np
from IPython.display import clear_output

varname = "tas_2m"  # User specifies: ["tas_2m", "pr_sfc"]
method = "allLeads"  # User specifies: ["allLeads", "weeks12lead", "weeks34lead", "weeks45lead"]
calculate_ensemble_mean = True  # User specifies: [True, False]
start_date = "1999-01-01" # User specifies: [eg, 1999-01-01] Note: doesn't need to be a monday
end_date = "2020-12-31"  # User specifies: [eg, 2020-12-31] Note: doesn't need to be a monday
destDir = "/glade/campaign/cesm/development/cross-wg/S2S/sglanvil/forJudith/" # User specifies 

# Check the time range if calculate_ensemble_mean is False
if not calculate_ensemble_mean:
    start = pd.to_datetime(start_date)
    end = pd.to_datetime(end_date)
    time_range = (end - start).days
    if time_range > 370:
        print(f"Warning: Your time range is {time_range} days, which exceeds the 370-day limit.")
        print("When choosing to keep all members, you need to shorten your date range to less than 370 days.")
        raise ValueError("Time range exceeds the allowed limit.")  # This will stop execution


base_dir = f"/glade/campaign/cesm/development/cross-wg/S2S/CESM2/S2SHINDCASTS/p1/{varname}"
mondays = pd.date_range(start_date, end_date, freq="W-MON")

methods = {
    "allLeads": (None, np.arange(1, 47)),  # No time slicing, set time coords
    "weeks12lead": (slice(0, 14), None),
    "weeks34lead": (slice(14, 28), None),
    "weeks56lead": (slice(28, 42), None),
}
time_slice, time_coords = methods.get(method, (None, None))

# Initialize result containers
data_results, init_array = [], []

for monday in mondays:
    short_date = monday.strftime("%d%b%Y").lower()
    init_date = monday.strftime("%Y%m%d")
    dir_path = os.path.join(base_dir, str(monday.year), f"{monday.month:02d}")
    clear_output(wait=True)  # Clears the current output
    print(short_date)
    if not os.path.exists(dir_path): 
        continue
    files = [os.path.join(dir_path, f) for f in os.listdir(dir_path) if f.endswith("m10.nc") and short_date in f]
    if not files:
        continue
    datasets = [xr.open_dataset(fp)[[varname]] for fp in files]
    combined = xr.concat(datasets, dim="member")

    # Apply time slicing if specified
    if time_slice:
        combined = combined.isel(time=time_slice).mean(dim="time")

    # Calculate or keep all ensemble members
    data = combined.mean(dim="member").expand_dims(init=[init_date]) if calculate_ensemble_mean else combined.expand_dims(init=[init_date])

    # Assign time coordinates if available
    if time_coords is not None:
        data = data.assign_coords({"time": time_coords})

    # Store results
    data_results.append(data[varname])
    init_array.append(init_date)

# Combine results across all dates
data_combined = xr.concat(data_results, dim="init")

# Create final dataset
lon_array, lat_array = combined.lon, combined.lat
dataset = xr.Dataset({
    "lon": lon_array,
    "lat": lat_array,
    "init": init_array,
    "data": data_combined,
})

dataset = dataset.assign_coords({"init": [pd.Timestamp(str(d)) for d in init_array]})

# Save raw concatenated data
dataset.to_netcdf(f"{destDir}/{varname}_cesm2cam6v2_allLeads_m10.nc")
dataset.to_zarr(f"{destDir}/{varname}_cesm2cam6v2_allLeads_m10.zarr", mode='w')

# Calculate and save climatology
climatology = dataset.groupby('init.dayofyear').mean()
climatology.to_netcdf(f"{destDir}/{varname}_clim_cesm2cam6v2_allLeads_m10.nc")

# Calculate and save anomalies
anomalies = dataset.groupby('init.dayofyear')-climatology
anomalies.to_netcdf(f"{destDir}/{varname}_anom_cesm2cam6v2_allLeads_m10.nc")


28dec2020


In [25]:
# init_dates = pd.to_datetime(dataset.init.values.astype(str), format='%Y%m%d')


In [ ]:
# init_dates = pd.to_datetime(dataset.init.values.astype(str), format='%Y%m%d')
# doy = init_dates.dayofyear.astype(int)
# doy = np.array(init_dates.dayofyear.astype(int))  # Convert to NumPy array
# is_leap_year = init_dates.is_leap_year
# shift_indices = (is_leap_year) & (doy > 59)  # March 1 is doy=60 in leap years
# doy[shift_indices] -= 1
# dataset = dataset.assign_coords(doy=("init", doy))
# climatology = dataset.groupby("doy").mean(dim="init", skipna=True)

# output_path = f"{destDir}/{varname}_clim_cesm2cam6v2_allLeads_m10.zarr"
# climatology.to_netcdf(fileOut)
